In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from models import CNN

Using TensorFlow backend.


In [2]:
# The data, split between train and test sets:
(X_train, y_train), (X, y) = cifar10.load_data()
print(X.shape)
print(y.shape)
print(np.min(X), np.max(X))

(10000, 32, 32, 3)
(10000, 1)
0 255


In [3]:
encoder = CNN(X.shape[1:])
_ = encoder(np.random.rand(1,32,32,3).astype(np.float32))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Baseline

In [15]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='auto').fit(np.reshape(X_train,(X_train.shape[0], -1)), y_train.squeeze())

/home/thallessilva/anaconda3/envs/tf2.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [17]:
print("Train score:", clf.score(np.reshape(X_train,(X_train.shape[0], -1)), y_train),
      "Test score:", clf.score(np.reshape(X_test,(X_test.shape[0], -1)), y_test))

Train score: 0.4815 Test score: 0.388


## Self-Supervised pre-training

In [7]:
X_train_repr = encoder(tf.cast(X_train, tf.float32) / 255, training=True)
X_test_repr = encoder(tf.cast(X_test, tf.float32) / 255, training=True)
print(X_train_repr.shape)
print(X_test_repr.shape)

(8000, 64)
(2000, 64)


In [8]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='auto').fit(X_train_repr, y_train.squeeze())

/home/thallessilva/anaconda3/envs/tf2.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [9]:
print(clf.score(X_train_repr, y_train.squeeze()), clf.score(X_test_repr, y_test.squeeze()))
# 0.333

0.39 0.372


In [10]:
X_train_repr[0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([0.93243945, 0.6365469 , 0.41792664, 1.3862063 , 1.0731233 ,
       1.35759   , 0.305511  , 1.1599735 , 0.7712196 , 1.261041  ,
       1.0972064 , 1.6589969 , 1.2036738 , 2.018743  , 0.6080824 ,
       0.84403247, 1.0010242 , 0.15506956, 1.2382021 , 1.1135885 ,
       0.59119797, 0.68139625, 1.1197722 , 1.1719528 , 0.82210064,
       1.1457936 , 0.9192853 , 0.8090557 , 1.5394404 , 1.1943905 ,
       1.6024731 , 1.3720269 , 0.89851224, 1.0545706 , 1.3131654 ,
       1.1160012 , 0.7884877 , 1.0297232 , 1.2930304 , 1.5905702 ,
       0.8161649 , 0.72504604, 0.9901402 , 0.5909488 , 1.2648938 ,
       1.2438143 , 0.9772537 , 1.8195769 , 0.4825426 , 0.84217083,
       0.91278833, 0.76893693, 1.7069005 , 1.0780828 , 1.1981698 ,
       1.0553076 , 1.112178  , 1.3993742 , 0.9017898 , 1.0375961 ,
       0.6001428 , 1.0199337 , 0.848011  , 1.2155435 ], dtype=float32)>

In [11]:
X_test_repr[0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([0.2842456 , 1.5738363 , 1.0443164 , 1.0045587 , 1.6872364 ,
       0.24549918, 1.1922263 , 1.0007584 , 1.2583567 , 0.68752915,
       0.9752594 , 1.100705  , 0.3946603 , 0.40060788, 1.0801685 ,
       0.547295  , 0.7615588 , 1.886446  , 0.60607046, 0.47095615,
       0.86944914, 0.9617727 , 0.8359903 , 0.9128037 , 0.80528295,
       0.44137582, 0.65994835, 0.4979187 , 1.0598176 , 0.4520391 ,
       0.30096218, 1.0096494 , 0.43420398, 0.4941259 , 0.98956835,
       0.1850791 , 0.3794615 , 0.7651825 , 0.37308517, 0.5678068 ,
       1.0759273 , 0.6956603 , 0.64980054, 0.8723744 , 0.5666764 ,
       0.9808809 , 2.1454136 , 0.16045755, 0.98682654, 1.2568831 ,
       1.2651138 , 0.16653773, 1.1673839 , 0.5723258 , 0.3958794 ,
       0.61943126, 0.33038038, 0.2741537 , 0.39372218, 0.9359734 ,
       1.2512028 , 0.9061063 , 1.0948234 , 0.71207774], dtype=float32)>

In [12]:
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
loss = cosine_loss([[0.0001, 1], [1., 1]], [[0.0002, 1], [0., 1]])

print('Loss: ', loss.numpy())  # Loss: 0.5

Loss:  -0.8535534


In [13]:
a = tf.convert_to_tensor([1,6,5], dtype=tf.int32)
b = tf.convert_to_tensor([4,3,2], dtype=tf.int32)
res = tf.sets.intersection(a[None,:],b[None,:])
print(tf.size(res))

tf.Tensor(0, shape=(), dtype=int32)


In [14]:
a = np.array([28.171087, 30.802778], dtype=np.float32)
b = np.array([2.2323670e-06, 2.1488113e-06], dtype=np.float32)

print(a / (a+b))

[0.99999994 0.99999994]
